In [ ]:
import sys
sys.path.append('./../../')

In [5]:
import ikamoana
import parcels
import numpy as np
import xarray as xr
import ikamoana.SEAPODYM_functions_Parcels3 as sfp

## __Initialize Habitat__

In [6]:
fh = ikamoana.feedinghabitat.FeedingHabitat(
    xml_filepath="./../../data/feeding_habitat_data/po_2x30_historical_interim/skj_po_interim_2deg_configuration_file.xml")

In [7]:
fh_test_field_set = fh.computeFeedingHabitat(0, time_end=0, lat_min=20, lat_max=23, lon_min=20, lon_max=23)
fh_test_field_set['Feeding_Habitat_Cohort_0'].data

array([[[0.00067908, 0.00073808, 0.00043648, 0.00037831],
        [0.00033385, 0.0003072 , 0.00031579, 0.00033039],
        [0.00030553, 0.00031155, 0.0003221 , 0.0003404 ],
        [0.00077659, 0.00096428, 0.00116014, 0.00134075]]], dtype=float32)

## __Add Ubathy__

In [15]:
# lmeso = lower meso
ubathy = fh.data_structure.variables_dictionary['forage_lmeso'].sel(fh_test_field_set.coords)

fh_test_field_set_to_merge = xr.Dataset(
    {'Ubathy':ubathy},
    coords=fh_test_field_set.coords,
)

final_test_field_set = fh_test_field_set.merge(fh_test_field_set_to_merge)
final_test_field_set

<xarray.Dataset>
Dimensions:                   (lon: 4, lat: 4, time: 1)
Coordinates:
  * lon                       (lon) float32 129.5 131.5 133.5 135.5
  * lat                       (lat) float32 24.5 22.5 20.5 18.5
  * time                      (time) datetime64[ns] 1979-01-15T12:00:00
Data variables:
    Feeding_Habitat_Cohort_0  (time, lat, lon) float32 0.0006791 ... 0.001341
    Ubathy                    (time, lat, lon) float32 1.522 1.949 ... 1.026
Attributes: (12/14)
    time_start:  None
    time_end:    0
    lat_min:     20
    lat_max:     23
    lon_min:     20
    lon_max:     23
    ...          ...
    sigma_K:     3.00000257690523
    T_star_1:    31.09996321152416
    T_star_K:    26.04960048814059
    bT:          2.83057972721579
    gamma:       0.0001000000005808978
    o_star:      3.764163854561827

## __Convert DataSet to FieldSet__

In [16]:
variables = {
    'H': 'Feeding_Habitat_Cohort_0',
    'Ubathy' : 'Ubathy'}

dimensions = {
    'lon': 'lon',
    'lat': 'lat',
    'time': 'time'}

field_set_test = parcels.FieldSet.from_xarray_dataset(
    final_test_field_set.reindex(lat=list(reversed(final_test_field_set.lat))),
    variables=variables,
    dimensions=dimensions)

## __Create Landmask__

In [17]:
landmask = sfp.Create_Landmask(field_set_test)
print("Landmask is :\n", landmask.data)

100% (4 of 4) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


Landmask is :
 [[[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]]


## __Compute Gradient__

In [18]:
gradient_dx_dy = sfp.getGradient(field=field_set_test.H, landmask=landmask)
print("\nDX is :\n",gradient_dx_dy[0].data[0])
print("\nDY is :\n",gradient_dx_dy[1].data[0])

Field for gradient calculation has time origin of 1979-01-15T12:00:00.000000000

DX is :
 [[ 8.9058205e-10  0.0000000e+00  0.0000000e+00  8.5692720e-10]
 [ 2.8919556e-11  3.9800181e-11  6.9292398e-11  8.7903997e-11]
 [-1.2976276e-10 -4.3979993e-11  5.6462765e-11  7.1122760e-11]
 [ 2.9178984e-10  0.0000000e+00  0.0000000e+00 -2.8766350e-10]]

DY is :
 [[-2.1195803e-09 -2.9370522e-09 -3.7708840e-09 -4.5011914e-09]
 [ 0.0000000e+00 -1.4783154e-09 -1.8996567e-09  0.0000000e+00]
 [ 0.0000000e+00  9.5961761e-10  2.5733737e-10  0.0000000e+00]
 [ 1.5534112e-09  1.9388138e-09  5.4310390e-10  2.1563236e-10]]


## __Compute Taxis__

In [19]:
taxis_tx_ty = sfp.Create_SEAPODYM_Taxis_Fields(
    dHdx=gradient_dx_dy[0], dHdy=gradient_dx_dy[1],
    length_classes=fh.data_structure.species_dictionary['cohorts_mean_length'],
    start_age=0)

print("Taxis (tx) is :\n", taxis_tx_ty[0].data[0])
print("Taxis (ty) is :\n", taxis_tx_ty[1].data[0])

Taxis for 1 steps
Taxis (tx) is :
 [[ 1.1729554e-05  0.0000000e+00  0.0000000e+00  1.1286297e-05]
 [ 3.7620990e-07  5.1775419e-07  9.0141378e-07  1.1435291e-06]
 [-1.6650082e-06 -5.6431486e-07  7.2448341e-07  9.1258835e-07]
 [ 3.6876020e-06  0.0000000e+00  0.0000000e+00 -3.6354540e-06]]
Taxis (ty) is :
 [[-2.9437495e-05 -4.0790837e-05 -5.2371397e-05 -6.2514169e-05]
 [ 0.0000000e+00 -2.0531375e-05 -2.6383115e-05  0.0000000e+00]
 [ 0.0000000e+00  1.3327515e-05  3.5739940e-06  0.0000000e+00]
 [ 2.1574335e-05  2.6926944e-05  7.5428225e-06  2.9947798e-06]]
